In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import optuna
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import time

# 设置随机种子以确保结果可重复
torch.manual_seed(42)
np.random.seed(42)

# -------------------
# 1. 数据加载和预处理
# -------------------
def load_data(years, data_dir):
    """加载 ICIO 表格数据"""
    data = []
    for year in years:
        file_path = os.path.join(data_dir, f"{year}_SML.csv")
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"文件 {file_path} 不存在，请检查路径或文件名")
        df = pd.read_csv(file_path, header=0, index_col=0)  # 第一行和第一列为表头              
        data.append(df.values)
    return np.stack(data, axis=0)  # 返回形状为 (num_years, num_rows, num_cols)


def preprocess_data(data):
    """数据预处理：展平并标准化"""
    num_years, num_rows, num_cols = data.shape
    input_dim = num_rows * num_cols
    data_flat = data.reshape(num_years, input_dim)  # (num_years, input_dim)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_flat)
    return torch.FloatTensor(data_scaled), scaler, input_dim

# 数据路径和年份
data_dir = "./ICIO DATA"  # 请替换为实际数据路径
years = list(range(2010, 2021))  # 1995-2020，共 26 年
data = load_data(years, data_dir)
data_tensor, scaler, input_dim = preprocess_data(data)

print(f"数据形状: {data.shape}")  # 应为 (26, num_rows, num_cols)
print(f"展平后数据形状: {data_tensor.shape}")  # 应为 (26, input_dim)
print(f"input_dim: {input_dim}")

# -------------------
# 2. Transformer 模型定义
# -------------------
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=nhead, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src):
        """前向传播"""
        memory = self.encoder(src)  # (batch_size, seq_len, input_dim)
        output = self.fc(memory[:, -1, :])  # 取最后一步输出，(batch_size, output_dim)
        return output

# -------------------
# 3. 数据准备函数
# -------------------
def prepare_sequences(data_tensor, seq_len):
    """生成时间序列数据"""
    X, y = [], []
    for i in range(len(data_tensor) - seq_len):
        X.append(data_tensor[i:i+seq_len])  # (seq_len, input_dim)
        y.append(data_tensor[i+seq_len])    # (input_dim)
    if len(X) == 0:
        raise ValueError(f"样本数为 0，请减少 seq_len（当前为 {seq_len}）或增加数据年份")
    return torch.stack(X), torch.stack(y)  # X: (num_samples, seq_len, input_dim), y: (num_samples, input_dim)

# -------------------
# 4. 超参数优化
# -------------------
def objective(trial):
    """Optuna 优化目标函数"""
    # 计算 input_dim 的所有因子
    factors = [i for i in range(1, input_dim + 1) if input_dim % i == 0]
    possible_nheads = [f for f in factors if f <= 8]  # 限制 nhead <= 8
    if not possible_nheads:
        possible_nheads = [1]  # 如果没有合适的因子，默认使用 1
    nhead = trial.suggest_categorical("nhead", possible_nheads)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)

    # 初始化模型
    model = TransformerModel(
        input_dim=input_dim,
        output_dim=input_dim,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # 准备序列数据
    seq_len = 5
    X, y = prepare_sequences(data_tensor, seq_len)
    num_samples = len(X)
    train_size = int(0.8 * num_samples)
    if train_size == 0:
        raise ValueError(f"训练集为空，num_samples={num_samples}，请减少 seq_len 或增加数据")
    print(f"num_samples: {num_samples}, train_size: {train_size}")

    train_X, val_X = X[:train_size], X[train_size:]
    train_y, val_y = y[:train_size], y[train_size:]
    train_dataset = TensorDataset(train_X, train_y)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    # 训练模型
    model.train()
    num_epochs = 10
    for epoch in range(num_epochs):
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()

    # 验证
    model.eval()
    with torch.no_grad():
        val_output = model(val_X)
        val_loss = criterion(val_output, val_y).item()
    return val_loss

# 运行超参数优化
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
best_params = study.best_params
print("最佳超参数:", best_params)

# -------------------
# 5. 使用最佳参数训练完整模型
# -------------------
seq_len = 5
X, y = prepare_sequences(data_tensor, seq_len)
train_size = int(0.8 * len(X))
train_X, val_X = X[:train_size], X[train_size:]
train_y, val_y = y[:train_size], y[train_size:]
train_dataset = TensorDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

model = TransformerModel(
    input_dim=input_dim,
    output_dim=input_dim,
    nhead=best_params["nhead"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]
)
optimizer = torch.optim.Adam(model.parameters(), lr=best_params["learning_rate"])
criterion = nn.MSELoss()

# TensorBoard 可视化
writer = SummaryWriter("runs/icio_transformer")

# 训练
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")
    writer.add_scalar("Loss/train", avg_loss, epoch)

# 保存模型
torch.save(model.state_dict(), "best_model.pth")

# -------------------
# 6. 预测未来 ICIO 表
# -------------------
model.eval()
future_years = 5  # 预测未来 5 年
predictions = []
last_sequence = data_tensor[-seq_len:].unsqueeze(0)  # (1, seq_len, input_dim)
with torch.no_grad():
    for _ in range(future_years):
        pred = model(last_sequence)  # (1, input_dim)
        predictions.append(pred.squeeze(0))
        last_sequence = torch.cat((last_sequence[:, 1:, :], pred.unsqueeze(1)), dim=1)

predictions = torch.stack(predictions)  # (future_years, input_dim)
predictions_np = scaler.inverse_transform(predictions.numpy())  # 反标准化
predictions_reshaped = predictions_np.reshape(future_years, data.shape[1], data.shape[2])

# 导出预测结果
for i, year in enumerate(range(2021, 2021 + future_years)):
    pd.DataFrame(predictions_reshaped[i], index=data[0].index, columns=data[0].columns).to_csv(f"pred_{year}.csv")

# -------------------
# 7. 可视化
# -------------------
plt.plot(range(1995, 2021), data_tensor[:, 0], label="Historical Data (first feature)")
plt.plot(range(2021, 2021 + future_years), predictions[:, 0], label="Predicted (first feature)")
plt.legend()
plt.title("Historical and Predicted ICIO Values")
plt.xlabel("Year")
plt.ylabel("Value")
plt.savefig("prediction_plot.png")
plt.show()

writer.close()

数据形状: (11, 3468, 3928)
展平后数据形状: torch.Size([11, 13622304])
input_dim: 13622304


RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2226805995220992 bytes.